In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types  import * 

In [ ]:
spark = SparkSession.builder.master("local").getOrCreate()

In [ ]:
df = spark.read.parquet("t_kdit_inversiones_master")

In [ ]:
df.printSchema()

root
 |-- user_name: string (nullable = true)
 |-- transaction_amount: decimal(9,2) (nullable = true)
 |-- main_account_id: string (nullable = true)
 |-- cutoff_date: date (nullable = true)



pregunta de clase

In [ ]:
#df.show(1)

In [ ]:
df1 = df.drop("transaction_amount")
df2 = df.select("transaction_amount")

In [ ]:
df11 = df1.withColumn("A",monotonically_increasing_id())
df22 = df2.withColumn("A",monotonically_increasing_id())

In [ ]:
dff3 =  df11.join(df22 , "A").drop("A")

In [ ]:
#dff3.show()

#### Filtrado

In [ ]:
#filter , where

In [ ]:
df.show(2)

+-----------------+------------------+---------------+-----------+
|        user_name|transaction_amount|main_account_id|cutoff_date|
+-----------------+------------------+---------------+-----------+
|Sylvester Sykes  |           9014.58|     1009014458| 2018-10-24|
|Brandon Wheeler  |           8590.93|     1006789262| 2018-10-24|
+-----------------+------------------+---------------+-----------+
only showing top 2 rows



In [ ]:
df.select("cutoff_date").distinct().show()

+-----------+
|cutoff_date|
+-----------+
| 2019-12-08|
| 2018-11-29|
| 2019-01-31|
| 2019-01-02|
| 2019-08-26|
| 2018-09-23|
| 2018-07-23|
| 2019-08-30|
| 2018-10-24|
| 2018-08-05|
| 2018-05-22|
+-----------+



In [ ]:
# pero tiene el mismo fin usar where

conditionExpression = (trim(col("user_name")) == "Sylvester Sykes")

df.filter(conditionExpression).show()

+-----------------+------------------+---------------+-----------+
|        user_name|transaction_amount|main_account_id|cutoff_date|
+-----------------+------------------+---------------+-----------+
|Sylvester Sykes  |           9014.58|     1009014458| 2018-10-24|
+-----------------+------------------+---------------+-----------+



In [ ]:
# help(df.where)
# help(df.filter)

In [ ]:

conditionExpression = (trim(col("user_name")) == "Sylvester Sykes")

df_A = df.filter(conditionExpression)


conditionExpression = (trim(col("user_name")) == "Brandon Wheeler")

df_B = df.filter(conditionExpression)

#### Union

In [ ]:
df_A.union(df_B).show()

+-----------------+------------------+---------------+-----------+
|        user_name|transaction_amount|main_account_id|cutoff_date|
+-----------------+------------------+---------------+-----------+
|Sylvester Sykes  |           9014.58|     1009014458| 2018-10-24|
|Brandon Wheeler  |           8590.93|     1006789262| 2018-10-24|
+-----------------+------------------+---------------+-----------+



#### Join

In [ ]:
df.show(2)

+-----------------+------------------+---------------+-----------+
|        user_name|transaction_amount|main_account_id|cutoff_date|
+-----------------+------------------+---------------+-----------+
|Sylvester Sykes  |           9014.58|     1009014458| 2018-10-24|
|Brandon Wheeler  |           8590.93|     1006789262| 2018-10-24|
+-----------------+------------------+---------------+-----------+
only showing top 2 rows



In [ ]:
df_default = df.select("main_account_id","transaction_amount").\
                withColumn("Mora", when(col("transaction_amount") > 90000, True).otherwise(False)).\
                    drop("transaction_amount")

In [ ]:
df_default.show(1)

+---------------+-----+
|main_account_id| Mora|
+---------------+-----+
|     1009014458|false|
+---------------+-----+
only showing top 1 row



In [ ]:
df.join(df_default, "main_account_id").show(4)

+---------------+-----------------+------------------+-----------+-----+
|main_account_id|        user_name|transaction_amount|cutoff_date| Mora|
+---------------+-----------------+------------------+-----------+-----+
|     1009014458|Sylvester Sykes  |           9014.58| 2018-10-24|false|
|     1009014458|Sylvester Sykes  |           9014.58| 2018-10-24|false|
|     1009014458|Sylvester Sykes  |           9014.58| 2018-10-24|false|
|     1009014458|Sylvester Sykes  |           9014.58| 2018-10-24|false|
+---------------+-----------------+------------------+-----------+-----+
only showing top 4 rows



Por si uno quiere corregir un tipo de dato especifico

In [ ]:
df_A.select(col("main_account_id").cast(IntegerType())).printSchema()

root
 |-- main_account_id: integer (nullable = true)



#### Crear Dataframe 

In [ ]:
data   = [("Javier", 34 ) ,("Carmen",27),("Kevin" , 28),("Ken",29) ]

schema = StructType([
                       StructField("name",StringType(),True), 
                       StructField("age",IntegerType(),True),  
                    ])


df = spark.createDataFrame(data,schema)

In [ ]:
df.show()

+------+---+
|  name|age|
+------+---+
|Javier| 34|
|Carmen| 27|
| Kevin| 28|
|   Ken| 29|
+------+---+



In [ ]:
df.groupBy("cutoff_date").sum().\
                          select("cutoff_date", col("sum(transaction_amount)").alias("suma")).\
                          show()

+-----------+---------+
|cutoff_date|     suma|
+-----------+---------+
| 2019-12-08| 25694.88|
| 2018-11-29|406670.45|
| 2019-01-31|621966.90|
| 2019-01-02|135241.45|
| 2019-08-26| 72015.79|
| 2018-09-23|924064.14|
| 2018-07-23| 72924.95|
| 2019-08-30|109537.16|
| 2018-10-24|995477.20|
| 2018-08-05| 73541.17|
| 2018-05-22|744399.23|
+-----------+---------+



### UDF  = "User Defined Funcion"

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
#"".join(list(reversed("marco")))

In [ ]:
fn_rev = lambda _ : _[::-1]

In [ ]:
fn_rev_mapper = udf(fn_rev)

In [ ]:
df.withColumn("dummy", fn_rev_mapper(col("user_name"))).show()

+-----------------+------------------+---------------+-----------+-----------------+
|        user_name|transaction_amount|main_account_id|cutoff_date|            dummy|
+-----------------+------------------+---------------+-----------+-----------------+
|Sylvester Sykes  |           9014.58|     1009014458| 2018-10-24|  sekyS retsevlyS|
|Brandon Wheeler  |           8590.93|     1006789262| 2018-10-24|  releehW nodnarB|
|Wing Roth        |          25660.92|     1025660292| 2018-10-24|        htoR gniW|
|Zachary Roth     |           9178.23|     1009178723| 2018-10-24|     htoR yrahcaZ|
|Otto Barron      |           6754.46|     1006754446| 2018-10-24|      norraB ottO|
|Gavin Mcleod     |          10447.75|     1010447375| 2018-10-24|     doelcM nivaG|
|Kyle Mckee       |          10657.29|     1009014458| 2018-10-24|       eekcM elyK|
|Nehru Woodward   |           8833.97|     1008833897| 2018-10-24|   drawdooW urheN|
|Ryan Wilson      |           8587.07|     1008587107| 2018-10-24